In [ ]:
from dataclasses import dataclass
from typing import Tuple, Callable, Dict, Optional, Iterable, List
from collections import deque

State = Tuple[int, int]  # (five_litre, three_litre)


In [ ]:
from typing import Tuple, Callable, Dict, Optional, Iterable, List
from collections import deque

State = Tuple[int, int]  # (five_litre, three_litre)


@dataclass(frozen=True)
class Capacities:
  # max capacity of each jug
    five: int = 5
    three: int = 3

class WaterJugPuzzle:
    def __init__(self, capacities: Capacities = Capacities()):
      # store jug cap
        self.cap = capacities
        # both start empty
        self.start: State = (0, 0)
        # mapping rules to their functions
        self.rules: Dict[str, Callable[[State], Optional[State]]] = {
            "fill_5": self.fill_5,
            "fill_3": self.fill_3,
            "empty_5": self.empty_5,
            "empty_3": self.empty_3,
            "pour_5_to_3": self.pour_5_to_3,
            "pour_3_to_5": self.pour_3_to_5,
        }
# makes sure they dont go over the cap
    def is_valid(self, s: State) -> bool:
        x, y = s
        return 0 <= x <= self.cap.five and 0 <= y <= self.cap.three
# defines the goal
    def goal(self, s: State) -> bool:
        return s[0] == 4
# makes sure the move is allowed
    def _guard(self, cond: bool, s: State) -> Optional[State]:
        return s if cond and self.is_valid(s) else None


# fill 5L jug
    def fill_5(self, s: State):
        return self._guard(s[0] < self.cap.five, (self.cap.five, s[1]))

# fill 3L
    def fill_3(self, s: State):
        return self._guard(s[1] < self.cap.three, (s[0], self.cap.three))

# empty 5L
    def empty_5(self, s: State):
        return self._guard(s[0] > 0, (0, s[1]))

# empty 3L
    def empty_3(self, s: State):
        return self._guard(s[1] > 0, (s[0], 0))

# pours 5 into 3 until 5 is empty or 3 is full
    def pour_5_to_3(self, s: State):
        x, y = s
        if x == 0 or y == self.cap.three:
            return None
        space = self.cap.three - y
        poured = min(x, space)
        return self._guard(True, (x - poured, y + poured))

#pours 3 to 5 until 3 is empty or 5 is full
    def pour_3_to_5(self, s: State):
        x, y = s
        if y == 0 or x == self.cap.five:
            return None
        space = self.cap.five - x
        poured = min(y, space)
        return self._guard(True, (x + poured, y - poured))



# generate all valid moves from current state
    def successors(self, s: State) -> Dict[str, State]:
        return {name: ns for name, rule in self.rules.items() if (ns := rule(s))}
# apply single rule to current state
    def apply_rule(self, s: State, rule_name: str) -> State:
        ns = self.rules[rule_name](s)
        if ns is None:
            raise ValueError(f"Illegal move: {rule_name} from {s}")
        return ns

 # apply sequence of rules from starting state
    def apply_sequence(self, seq: Iterable[str], start: Optional[State]=None, verbose=True) -> State:
        s = self.start if start is None else start
        if verbose:
            print(f"Start: {s}")
        for i, r in enumerate(seq, 1):
            s = self.apply_rule(s, r)
            if verbose:
                print(f"{i}. {r} -> {s}")
        if verbose:
            print(f"End: {s} {'(GOAL)' if self.goal(s) else ''}")
        return s


# breadth first to find the shortest sequence
    def bfs(self) -> List[str]:
        frontier = deque([self.start])
        parent = {self.start: None}
        parent_rule = {self.start: None}
        while frontier:
            s = frontier.popleft()
            # re create path of rule names from goal to start
            if self.goal(s):
                path, cur = [], s
                while parent[cur] is not None:
                    path.append(parent_rule[cur])
                    cur = parent[cur]
                return list(reversed(path))
            for rule_name, ns in self.successors(s).items():
                if ns not in parent:
                    parent[ns] = s
                    parent_rule[ns] = rule_name
                    frontier.append(ns)
        raise RuntimeError("No solution")

In [ ]:
puzzle = WaterJugPuzzle()

sequence = [
    "fill_5",
    "pour_5_to_3",
    "empty_3",
    "pour_5_to_3",
    "fill_5",
    "pour_5_to_3"
]

final_state = puzzle.apply_sequence(sequence)
assert puzzle.goal(final_state)



Start: (0, 0)
1. fill_5 -> (5, 0)
2. pour_5_to_3 -> (2, 3)
3. empty_3 -> (2, 0)
4. pour_5_to_3 -> (0, 2)
5. fill_5 -> (5, 2)
6. pour_5_to_3 -> (4, 3)
End: (4, 3) (GOAL)


In [ ]:
auto_seq = puzzle.bfs()
print("Shortest sequence found by BFS:", auto_seq)


Shortest sequence found by BFS: ['fill_5', 'pour_5_to_3', 'empty_3', 'pour_5_to_3', 'fill_5', 'pour_5_to_3']
